In [1]:
from diel_models.pipeline import Pipeline, PhotonRestrictor, Compartments, Biomass, NitrateUptake, StoragePool
import os
import cobra
from tests import TEST_DIR

aragem_model_path = os.path.join(TEST_DIR, 'data', 'AraGEM2010.xml')

modelo = cobra.io.read_sbml_model(aragem_model_path)

Steps = [
    Compartments(modelo),
    StoragePool(modelo, metabolites = ["S_Sucrose_c_Day", "S_Sulfate_c_Day", "S_Nitrate_c_Day", "S_L_45_Histidine_c_Day", "S_L_45_Isoleucine_c_Day", "S_L_45_Leucine_c_Day", "S_L_45_Lysine_c_Day", "S_L_45_Methionine_c_Day", "S_L_45_Phenylalanine_c_Day", "S_L_45_Threonine_c_Day", "S_L_45_Tryptophan_c_Day", "S_L_45_Valine_c_Day", "S_L_45_Arginine_c_Day", "S_L_45_Cysteine_c_Day", "S_L_45_Glutamine_c_Day", "S_L_45_Glutamate_c_Day", "S_Glycine_c_Day", "S_L_45_Proline_c_Day", "S_L_45_Tyrosine_c_Day", "S_L_45_Alanine_c_Day", "S_L_45_Asparagine_c_Day", "S_L_45_Serine_c_Day", "S_Orthophosphate_c_Day", "S_Starch_p_Day", "S_D_45_Fructose_c_Day", "S__40_S_41__45_Malate_c_Day", "S_Fumarate_c_Day", "S_Citrate_c_Day"]),
    PhotonRestrictor(modelo, "Ex16_Night"),
    Biomass(modelo, "BIO_L_Day", "BIO_L_Night", ["R03845_c_Night", "R06267_c_Night", "R06286_c_Night", "R04801_c_Night", "R07856_c_Night", "R04803_c_Night", "R06961_c_Night", "R06960_c_Night", "V0004_Night", "R06284_c_Night", "R05618_c_Night", "R03824_c_Night", "R05341_c_Night", "R06965_c_Night", "R07558_c_Night", "R04798_c_Night", "R04787_c_Night", "R07840_c_Night", "R06963_c_Night", "R06962_c_Night", "R07850_c_Night", "R07530_c_Night"]),
    NitrateUptake(modelo, "Ex4_Day", "Ex4_Night")
]

pipeline = Pipeline(modelo, Steps)
pipeline.run()

In [2]:
modelo.reactions.get_by_id("Biomass_Total").upper_bound = 0.11
modelo.reactions.get_by_id("Biomass_Total").lower_bound = 0.11
modelo.objective = "Ex16_Day"
modelo.objective_direction = "max"

In [4]:
from cobra.flux_analysis import pfba

solution = pfba(modelo)
R02732 = solution.fluxes["R02732_c_Day"]
R01070N = solution.fluxes["R01070N_c_Day"]
R00762 = solution.fluxes["R00762_c_Day"]
chex2 = round(((R02732 - R01070N) + R00762) / 2, 2)
print(chex2)

9.31


In [6]:
R04779 = solution.fluxes["R04779_p_Day"]
R00762 = solution.fluxes["R00762_p_Day"]
R01070N = solution.fluxes["R01070N_p_Day"]

phex1 = round(((R04779 - R00762) + R01070N) / 2, 2)
phex1

-39.42